In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader

from torchinfo import summary
from models import XSwinFusion 
from pose_estimation import PoseDataNPZTorch


WORKDIR = f"{os.getcwd()}/.."
DATA_FOLDER = os.path.join(WORKDIR, "data_folder")
DATASET_NPZ_PATH = os.path.join(DATA_FOLDER, "dataset_npz")


In [2]:
x = torch.tensor([0, 1, 2, 3, 4, 5])

x = x[[2,3, 2,3,2 ,3 ,2 ,3]]
x


tensor([2, 3, 2, 3, 2, 3, 2, 3])

In [3]:
samples = 1_000
resize = (432, 768)
aspect_ratio = True
margin = 12
feature_dims = 64

fusion = XSwinFusion(num_points=samples, feature_dims=feature_dims, resize=resize)

dataset = None
dataset = PoseDataNPZTorch(DATASET_NPZ_PATH, samples=samples, 
                           resize=resize, aspect_ratio=aspect_ratio, margin=margin)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

summary(fusion, depth=1)


Presumed Preloaded NPZ Dataset: /Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz


Layer (type:depth-idx)                                            Param #
XSwinFusion                                                       --
├─XNetSwinTransformer: 1-1                                        9,822,604
├─PointNet: 1-2                                                   622,482
├─Sequential: 1-3                                                 12,672
├─Sequential: 1-4                                                 265,088
├─Sequential: 1-5                                                 3,161
├─Sequential: 1-6                                                 2,883
├─Sequential: 1-7                                                 2,865
Total params: 10,731,755
Trainable params: 10,641,634
Non-trainable params: 90,121

In [4]:
R = torch.eye(3)
T = torch.ones((3, 1))
print(R.shape, T.shape)
torch.cat((R, T), dim=-1)


torch.Size([3, 3]) torch.Size([3, 1])


tensor([[1., 0., 0., 1.],
        [0., 1., 0., 1.],
        [0., 0., 1., 1.]])

In [6]:
points = []

for i, (s, t, c, mi, p) in enumerate(dataloader):

    x, transforms = fusion(t, c, mi)
    print(s.shape)
    print(t.shape)
    print(x.shape)

    R = x[:, :, :3]
    T = x[:, :, 3]

    x = torch.bmm(s, R.transpose(-2, -1)) + T.unsqueeze(-2)
    print(x.shape)

    if i == 0: 
        assert 0


torch.Size([2, 3, 4])
torch.Size([2, 1000, 3])
torch.Size([2, 1000, 3])
torch.Size([2, 3, 4])
torch.Size([2, 1000, 3])


AssertionError: 